In [1]:
import pandas as pd
import requests
import zipfile
import io
import plotly.graph_objects as go
from ipywidgets import widgets

Get the data from gstatic

In [2]:
filename='2020_GB_Region_Mobility_Report.csv'

def _get_data():
    zip_file_url = 'https://www.gstatic.com/covid19/mobility/Region_Mobility_Report_CSVs.zip'
    r = requests.get(zip_file_url)
    z = zipfile.ZipFile(io.BytesIO(r.content))
    z.extractall(members=[filename])

In [3]:
# only run if need to update
_get_data()

Create data frame, drop any unneeded columns

In [4]:
drop_columns=['country_region_code','country_region','metro_area','iso_3166_2_code', 'census_fips_code']
df = pd.read_csv(filename).drop(drop_columns, axis=1)
df

,sub_region_1,sub_region_2,date,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline
0,NaN,NaN,2020-02-15,-12.0,-7.0,-35.0,-12.0,-4.0,2.0
1,NaN,NaN,2020-02-16,-7.0,-6.0,-28.0,-7.0,-3.0,1.0
2,NaN,NaN,2020-02-17,10.0,1.0,24.0,-2.0,-14.0,2.0
3,NaN,NaN,2020-02-18,7.0,-1.0,20.0,-3.0,-14.0,2.0
4,NaN,NaN,2020-02-19,6.0,-2.0,8.0,-4.0,-14.0,3.0
...,...,...,...,...,...,...,...,...,...
138394,York,NaN,2021-01-08,-74.0,-31.0,-63.0,-73.0,-56.0,25.0
138395,York,NaN,2021-01-09,-76.0,-32.0,-55.0,-76.0,-41.0,17.0
138396,York,NaN,2021-01-10,-78.0,-32.0,-50.0,-77.0,-43.0,14.0
138397,York,NaN,2021-01-11,-67.0,-29.0,-44.0,-71.0,-57.0,22.0


make the widgets

In [5]:
chart_options = widgets.Dropdown(
    options=list(df.columns[3:]),
    value='retail_and_recreation_percent_change_from_baseline',
    description='Mobility area'
)

sub_region_1_options = widgets.Dropdown(
    options=list(df['sub_region_1'].unique()),
    value='Greater London',
    description='Sub region 1',
)

def _get_sr2_options():
    return list(df[df['sub_region_1']==sub_region_1_options.value]['sub_region_2'].unique())

sub_region_2_options = widgets.Dropdown(
    options=_get_sr2_options(),
    value='London Borough of Barnet',
    description='Sub region 2',
)

In [6]:
def get_x_and_y():
    filter_list = [i and j for i, j in
                    zip(df['sub_region_1'] == sub_region_1_options.value,
                        df['sub_region_2'] == sub_region_2_options.value)]
    temp_df = df[filter_list]
    x = temp_df['date']
    y = temp_df[chart_options.value]
    return [x, y]

In [7]:
def get_title():
    return sub_region_2_options.value+ '<br>' + chart_options.value

In [8]:
# Assign an empty figure widget
x, y = get_x_and_y()
trace = go.Scatter(x=x, y=y, name=chart_options.value)
g = go.FigureWidget(data=[trace],
                    layout=go.Layout(
                        title=dict(
                            text=get_title(),
                            x=0.5
                        ),
                        yaxis=dict(
                            title={'text': chart_options.value}
                        )
                    ))

In [9]:
# write a function that will handle the input from the widgets, and alter the state of the graph.

def response(change):
    x, y = get_x_and_y()
    sub_region_2_options.options=_get_sr2_options()
    with g.batch_update():
        g.data[0].x = x
        g.data[0].y = y
        g.layout.title=dict(text=get_title(), x=0.5)
        g.layout.yaxis.title = chart_options.value

chart_options.observe(response, names="value")
sub_region_1_options.observe(response, names="value")
sub_region_2_options.observe(response, names="value")

In [10]:
opts = widgets.VBox([chart_options, sub_region_1_options, sub_region_2_options])
widg = widgets.VBox([opts, g])
widg

In [11]:
# from ipywidgets.embed import embed_minimal_html
# embed_minimal_html('index.html', views=[widg], title='Widgets export')